# Laboratório 10: Usar o SageMaker Pipelines e o SageMaker Model Registry com o SageMaker Studio

Neste laboratório, você vai criar e executar um Amazon SageMaker Pipeline e monitorar o progresso do pipeline. Você também vai localizar e explorar alguns artefatos que o processo de machine learning (ML) usa ou gera.

Se houver tempo, você também poderá examinar os detalhes de linhagem do modelo que o pipeline gerou.

## Tarefa 2.1 Configuração do ambiente

Antes de criar o pipeline do SageMaker, você precisa preparar o ambiente instalando os pacotes necessários, importando módulos e preparando os arquivos de suporte. Este pipeline foi projetado para usar um grupo de recursos, portanto, você também vai criar um grupo de recursos no Amazon SageMaker Feature Store e executar um fluxo do Data Wrangler para preparar o ambiente. 

Execute as células nesta tarefa para fazer o seguinte:
- Instalar as dependências.
- Importar os módulos necessários.
- Copiar os dados e o código no Amazon Simple Storage Service (Amazon S3).
- Criar um grupo de recursos.
- Fazer a ingestão dos recursos no grupo de recursos.

### Tarefa 2.1.1: Instalar dependências

In [ ]:
#install dependencies
%pip install --upgrade pip 
%pip install pytest-astropy ==  0.7.0
%pip install rsa == 4.7.2
%pip install PyYAML
!apt update && apt install -y git
%pip install git+https://github.com/aws-samples/ml-lineage-helper

### Tarefa 2.1.2 Importar módulos

In [ ]:
#import-modules
import os
import json
import boto3
import sagemaker
import sagemaker_datawrangler
import sagemaker.session
import datetime as dt
import pandas as pd
import time
from time import gmtime, strftime
import uuid
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.transformer import Transformer
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from ml_lineage_helper import *
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor
from sagemaker.network import NetworkConfig
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

In [ ]:
#create sessions
boto_session  =  boto3.Session()
sagemaker_session = sagemaker.Session()


In [ ]:
#create clients
s3_client = boto3.client('s3')
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')
sagemaker_client = boto3.client('sagemaker')

In [ ]:
#feature store session
feature_store_session = Session(
    boto_session = boto_session,
    sagemaker_client = sagemaker_client,
    sagemaker_featurestore_runtime_client = featurestore_runtime
)

In [ ]:
#set global variables
default_bucket = sagemaker_session.default_bucket()
region = boto_session.region_name
role = sagemaker.get_execution_role()

### Tarefa 2.1.3 Copiar arquivos de laboratório no Amazon S3 

In [ ]:
# Upload files to default bucket
s3_client.put_object(Bucket = default_bucket, Key = 'data/')
s3_client.put_object(Bucket = default_bucket, Key = 'input/code/')
s3_client.upload_file('pipelines/data/storedata_total.csv', default_bucket, 'data/storedata_total.csv')
s3_client.upload_file('pipelines/input/code/evaluate.py', default_bucket, 'input/code/evaluate.py')
s3_client.upload_file('pipelines/input/code/generate_config.py', default_bucket, 'input/code/generate_config.py')
s3_client.upload_file('pipelines/input/code/processfeaturestore.py', default_bucket, 'input/code/processfeaturestore.py')

# Preview the dataset
print('Dataset preview:')
customer_data = pd.read_csv('pipelines/data/storedata_total.csv')
customer_data.head()

### Tarefa 2.1.4 Criar o grupo de recursos

Nesta tarefa, você vai criar um grupo de recursos para os dados. Primeiro, crie um esquema dos dados. Neste laboratório, o esquema deve ocorrer pelas colunas **name** e depois pelo **type** de variável.

In [ ]:
#set-up-feature-store-variables
record_identifier_feature_name = 'FS_ID'
event_time_feature_name = 'FS_time'

column_schemas = [
    {
        "name": "retained",
        "type": "long"
    },
    {
        "name": "esent",
        "type": "long"
    },
    {
        "name": "eopenrate",
        "type": "float"
    },
    {
        "name": "eclickrate",
        "type": "float"
    },
    {
        "name": "avgorder",
        "type": "float"
    },
    {
        "name": "ordfreq",
        "type": "float"
    },
    {
        "name": "paperless",
        "type": "long"
    },
    {
        "name": "refill",
        "type": "long"
    },
    {
        "name": "doorstep",
        "type": "long"
    },
    {
        "name": "first_last_days_diff",
        "type": "long"
    },
    {
        "name": "created_first_days_diff",
        "type": "long"
    },
    {
        "name": "favday_Friday",
        "type": "long"
    },
    {
        "name": "favday_Monday",
        "type": "long"
    },
    {
        "name": "favday_Saturday",
        "type": "long"
    },
    {
        "name": "favday_Sunday",
        "type": "long"
    },
    {
        "name": "favday_Thursday",
        "type": "long"
    },
    {
        "name": "favday_Tuesday",
        "type": "long"
    },
    {
        "name": "favday_Wednesday",
        "type": "long"
    },
    {
        "name": "city_BLR",
        "type": "long"
    },
    {
        "name": "city_BOM",
        "type": "long"
    },
    {
        "name": "city_DEL",
        "type": "long"
    },
    {
        "name": "city_MAA",
        "type": "long"
    },
    {
        "name": "FS_ID",
        "type": "long"
    },
    {
        "name": "FS_time",
        "type": "float"
    }
]


Agora, crie o grupo de recursos.

In [ ]:
# Flow name and a unique ID for this export (used later as the processing job name for the export)
flow_name = 'featureengineer'
flow_export_id = f"{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

# Feature group name, with flow_name and a unique id. You can give it a customized name
feature_group_name = f"FG-{flow_name}-{str(uuid.uuid4())[:8]}"

# SageMaker Feature Store writes the data in the offline store of a Feature Group to a 
# Amazon S3 location owned by you.
feature_store_offline_s3_uri = 's3://' + default_bucket

# Controls if online store is enabled. Enabling the online store allows quick access to 
# the latest value for a record by using the GetRecord API.
enable_online_store = True

In [ ]:
#create-feature-group
default_feature_type = FeatureTypeEnum.STRING
column_to_feature_type_mapping = {
    "float": FeatureTypeEnum.FRACTIONAL,
    "long": FeatureTypeEnum.INTEGRAL
}

feature_definitions = [
    FeatureDefinition(
        feature_name = column_schema['name'], 
        feature_type = column_to_feature_type_mapping.get(column_schema['type'], default_feature_type)
    ) for column_schema in column_schemas
]


print(f"Feature Group Name: {feature_group_name}")

# Confirm the Athena settings are configured
try:
    boto3.client('athena').update_work_group(
        WorkGroup = 'primary',
        ConfigurationUpdates = {
            'EnforceWorkGroupConfiguration':False
        }
    )
except Exception:
    pass

feature_group = FeatureGroup(
    name = feature_group_name, sagemaker_session = feature_store_session, feature_definitions = feature_definitions)

feature_group.create(
    s3_uri = feature_store_offline_s3_uri,
    record_identifier_name = record_identifier_feature_name,
    event_time_feature_name = event_time_feature_name,
    role_arn = role,
    enable_online_store = enable_online_store
)

def wait_for_feature_group_creation_complete(feature_group):
    """Helper function to wait for the completions of creating a feature group"""
    response = feature_group.describe()
    status = response.get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for feature group creation")
        time.sleep(5)
        response = feature_group.describe()
        status = response.get("FeatureGroupStatus")

    if status != "Created":
        print(f"Failed to create feature group, response: {response}")
        failureReason = response.get("FailureReason", "")
        raise SystemExit(
            f"Failed to create feature group {feature_group.name}, status: {status}, reason: {failureReason}"
        )
    print(f"Feature Group {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(feature_group = feature_group)


### Tarefa 2.1.5 Ingerir recursos

Este processo exige cerca de cinco minutos para ser concluído.

In [ ]:
#populate-feature-store
column_list = ['retained','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday', 'favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA','FS_ID','FS_time']
lab_test_data = pd.read_csv('featureengineer_data/store_data_processed.csv', names = (column_list), header = 1)
feature_group.ingest(data_frame = lab_test_data, wait = True)

## Tarefa 2.2 Criar e executar um pipeline do SageMaker

Agora que o ambiente está definido, você pode configurar, criar e iniciar um pipeline do SageMaker. 

Um pipeline do SageMaker é um fluxo de trabalho que executa uma série de etapas dependentes. As etapas podem aceitar entradas e enviar saídas, para que dados e outros ativos possam ser passados entre elas. 

Execute as seguintes células para:
- Definir as variáveis necessárias para configurar o pipeline.
- Configurar uma sessão do SageMaker.
- Definir as etapas do pipeline.
- Configurar o pipeline.
- Criar o pipeline.
- Iniciar o pipeline.
- Descrever o pipeline.
- Criar um evento de espera para que o notebook não continue até que a execução do pipeline termine.

### Tarefa 2.2.1 Configurar as variáveis que o pipeline usa

In [ ]:

#pipeline-variables
feature_group_name = feature_group.name
model_name = "Churn-model"

sklearn_processor_version = "0.23-1"
model_package_group_name = "ChurnModelPackageGroup"
pipeline_name = "ChurnModelSMPipeline"

processing_instance_count = ParameterInteger(
    name = "ProcessingInstanceCount",
    default_value = 1
    )

processing_instance_type = ParameterString(
        name = "ProcessingInstanceType",
        default_value = "ml.m5.xlarge"
    )

training_instance_type = ParameterString(
        name = "TrainingInstanceType",
        default_value = "ml.m5.xlarge"
    )

input_data = ParameterString(
        name = "InputData",
        default_value = "s3://{}/data/storedata_total.csv".format(default_bucket), 
    )

batch_data = ParameterString(
        name = "BatchData",
        default_value = "s3://{}/data/batch/batch.csv".format(default_bucket),
    )

### Tarefa 2.2.2 Configurar o pipeline

Defina um pipeline chamado **ChurnModelPipeline** para produzir um modelo que avalie a probabilidade de reter ou manter clientes. Este pipeline tem nove etapas. 

Cada etapa em um pipeline executa um tipo de trabalho específico. As entradas necessárias para um trabalho variam de acordo com o tipo de trabalho. Consulte [Tipos de etapa](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#build-and-manage-steps-types) para saber mais sobre os tipos de etapa de pipeline do SageMaker.

Examine o código nas seguintes células para entender como cada etapa foi definida:

A etapa **ChurnModelProcess** é definida na variável chamada **step_process**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): processamento – os trabalhos de processamento são definidos usando a classe ProcessingStep().
- **Processor** (Processador: SKLearnProcessor.
- **Destination** (Destino): a saída será enviada às pastas no bucket do S3 padrão.
- **Job Arguments** (Argumentos do trabalho): esta etapa usará o Feature Store para processar o conjunto de dados.
- **Code** (Código): **processfeaturestore.py**, que reside no bucket do S3 padrão.


In [ ]:
#configure-processing-step
# Run a scikit-learn script to do data processing on SageMaker 
# using the SKLearnProcessor class
sklearn_processor = SKLearnProcessor(
        framework_version = sklearn_processor_version,
        instance_type = processing_instance_type.default_value, 
        instance_count = processing_instance_count,
        sagemaker_session = sagemaker_session,
        role = role,
    )

# Inputs, outputs, and code are parameters to the processor
# step_* will become the pipeline steps toward the end of the cell
# in this case, use the feature store as input, so there is no externalinput
step_process = ProcessingStep(
        name = "ChurnModelProcess",
        processor = sklearn_processor,
        outputs = [
            ProcessingOutput(output_name = "train", source = "/opt/ml/processing/train",\
                             destination = f"s3://{default_bucket}/output/train" ),
            ProcessingOutput(output_name = "validation", source = "/opt/ml/processing/validation",\
                            destination = f"s3://{default_bucket}/output/validation"),
            ProcessingOutput(output_name = "test", source = "/opt/ml/processing/test",\
                            destination = f"s3://{default_bucket}/output/test"),
            ProcessingOutput(output_name = "batch", source = "/opt/ml/processing/batch",\
                            destination = f"s3://{default_bucket}/data/batch"),
            ProcessingOutput(output_name = "baseline", source = "/opt/ml/processing/baseline",\
                            destination = f"s3://{default_bucket}/input/baseline")
        ],
        job_arguments = ["--featuregroupname",feature_group_name,"--default-bucket",default_bucket,"--region",region],
        code = f"s3://{default_bucket}/input/code/processfeaturestore.py",
    )

A etapa **ChurnHyperParameterTuning** é definida na variável chamada **step_tuning**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): ajuste – os trabalhos de ajuste são definidos usando a classe TuningStep().
- **Tuner** (Ajustador): este trabalho usa o framework XGBoost.
- **Entradas:** observe que este trabalho usa os dados de treinamento e validação produzidos pela etapa ChurnModelProcess, **step_process**.

In [ ]:
#configure-churn-hyperparameter-tuning
# Training/tuning step for generating model artifacts
model_path = f"s3://{default_bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework = "xgboost",
    region = region,
    version = "1.5-1",
    py_version = "py3",
    instance_type = training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
    }

xgb_train = Estimator(
    image_uri = image_uri,
    instance_type = training_instance_type,
    instance_count = 1,
    hyperparameters = fixed_hyperparameters,
    output_path = model_path,
    base_job_name = f"churn-train",
    sagemaker_session = sagemaker_session,
    role = role
    )

In [ ]:
#Tuning steps
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
    }
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "ChurnHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs = 2, max_parallel_jobs = 2),
    inputs = {
            "train": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
            "validation": TrainingInput(
                s3_data = step_process.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type = "text/csv",
            ),
        },
    )

A etapa **ChurnEvalBestModel** é definida na variável chamada **step_eval**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): processamento.
- **Processor** (Processador): ScriptProcessor.
- **Inputs** (Entradas): observe que este trabalho usa o modelo principal de ChurnHyperParameterTuning (**step_tuning**) e a saída de teste de ChurnModelProcess (**step_process**).
- **Outputs** (Saídas): a saída é gravada no bucket do S3 padrão.
- **Code** (Código): um script chamado **evaluate.py**, que reside no Amazon S3, é usado para avaliação.

In [ ]:
#configure-churn-best-model
evaluation_report = PropertyFile(
    name = "ChurnEvaluationReport",
    output_name = "evaluation",
    path = "evaluation.json",
)

script_eval = ScriptProcessor(
    image_uri = image_uri,
    command = ["python3"],
    instance_type = processing_instance_type,
    instance_count = 1,
    base_job_name = "script-churn-eval",
    role = role,
    sagemaker_session = sagemaker_session,
)

step_eval = ProcessingStep(
    name = "ChurnEvalBestModel",
    processor = script_eval,
    inputs = [
        ProcessingInput(
            source = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
            destination = "/opt/ml/processing/model"
        ),
        ProcessingInput(
            source = step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination = "/opt/ml/processing/test"
        )
    ],
    outputs = [
        ProcessingOutput(output_name = "evaluation", source = "/opt/ml/processing/evaluation",\
                            destination = f"s3://{default_bucket}/output/evaluation"),
    ],
    code = f"s3://{default_bucket}/input/code/evaluate.py",
    property_files = [evaluation_report],
)

A etapa **ChurnCreateModel** é definida na variável chamada **step_create_model**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): modelo – os trabalhos de modelo são definidos usando a classe Model().
- **Model** (Modelo): o modelo usado pela etapa é definido na variável já definida chamada **model**. Observe que a variável **model** usa o modelo principal que foi criado por ChurnHyperParameterTuning (**step_tuning**).
- **Inputs** (Entradas): as entradas incluem um tipo de instância e um tipo de acelerador.

In [ ]:
#configure-model-creation
model = Model(
    image_uri = image_uri,        
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0,s3_bucket = default_bucket,prefix = "output"),
    name = model_name,
    sagemaker_session = sagemaker_session,
    role = role,
)

inputs = CreateModelInput(
    instance_type = "ml.m5.large",
    accelerator_type = "ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name = "ChurnCreateModel",
    model = model,
    inputs = inputs,
)

A etapa **ChurnModelConfigFile** é definida na variável chamada **step_config_file**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): processamento.
- **Processor** (Processador): ScriptProcessor.
- **Code** (Código): **generate_config.py**, que reside no bucket do S3 padrão.
- **Job Arguments** (Argumentos do trabalho): os argumentos do trabalho incluem o modelo que foi gerado por **ChurnCreateModel**, o caminho para o relatório de viés, o bucket padrão, o número de amostras e o número de instâncias usadas no processamento.
- **Depends On** (Depende de): observe que este trabalho só pode ser executado quando a criação do modelo é concluída.

In [ ]:

#configure-script-processing
bias_report_output_path = f"s3://{default_bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
analysis_config_path = f"s3://{default_bucket}/clarify-output/bias/analysis_config.json"
clarify_image = sagemaker.image_uris.retrieve(framework = 'sklearn', version = sklearn_processor_version, region = region)

#custom_image_uri = None
script_processor = ScriptProcessor(
    command = ['python3'],
    image_uri = clarify_image,
    role = role,
    instance_count = 1,
    instance_type = processing_instance_type,
    sagemaker_session = sagemaker_session,
)

step_config_file = ProcessingStep(
    name = "ChurnModelConfigFile",
    processor = script_processor,
    code = f"s3://{default_bucket}/input/code/generate_config.py",
    job_arguments = ["--modelname", step_create_model.properties.ModelName, "--bias-report-output-path", bias_report_output_path, "--clarify-instance-type", clarify_instance_type,\
                  "--default-bucket", default_bucket, "--num-baseline-samples", "50", "--instance-count", "1"],
    depends_on = [step_create_model.name]
)

A etapa **ChurnTransform** é definida na variável chamada **step_transform**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): transformação – os trabalhos de transformação são definidos usando a classe TransformStep().
- **Transformer** (Transformador): os detalhes do transformador são definidos na variável já definida chamada **transformer**. Observe que essa variável está usando o modelo que foi criado em ChurnCreateModel (**step_create_model**).
- **Inputs** (Entradas): os dados que serão transformados, batch.csv, que já foram definidos no notebook. A entrada também inclui o tipo de arquivo e como ele deve ser dividido.

In [ ]:
#configure-inference
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type = "ml.m5.xlarge",
    instance_count = 1,
    assemble_with = "Line",
    accept = "text/csv",    
    output_path = f"s3://{default_bucket}/ChurnTransform"
    )

step_transform = TransformStep(
    name = "ChurnTransform",
    transformer = transformer,
    inputs = TransformInput(data = batch_data, content_type = "text/csv", join_source = "Input", split_type = "Line")
    )

A etapa **ClarifyProcessingStep** é definida na variável chamada **step_clarify**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): processamento.
- **Processor** (Processador): este trabalho usa SageMakerClarifyProcessor. É possível examinar a configuração do processador na variável chamada **clarify_processor**.
- **Inputs** (Entradas): as entradas são definidas nas variáveis **data_input** e **congif_input**.
- **Outputs** (Saídas): a saída é gravada em uma pasta no bucket padrão. 
- **Depends On** (Depende de): observe que este trabalho só pode ser executado depois que o arquivo de configuração exigido pelo Amazon SageMaker Clarify é criado por **ChurnModelConfigFile**. 

In [ ]:
#configure-clarify-processing
data_config = sagemaker.clarify.DataConfig(
s3_data_input_path = f's3://{default_bucket}/output/train/train.csv',
s3_output_path = bias_report_output_path,
    label = 0,
    headers = ['target','esent','eopenrate','eclickrate','avgorder','ordfreq','paperless','refill','doorstep','first_last_days_diff','created_first_days_diff','favday_Friday','favday_Monday','favday_Saturday','favday_Sunday','favday_Thursday','favday_Tuesday','favday_Wednesday','city_BLR','city_BOM','city_DEL','city_MAA'],
    dataset_type = "text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role = role,
    instance_count = 1,
    instance_type = clarify_instance_type,
    sagemaker_session = sagemaker_session,
)

config_input = ProcessingInput(
    input_name = "analysis_config",
    source=analysis_config_path,
    destination = "/opt/ml/processing/input/analysis_config",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_compression_type = "None",
    )

data_input = ProcessingInput(
    input_name = "dataset",
    source = data_config.s3_data_input_path,
    destination = "/opt/ml/processing/input/data",
    s3_data_type = "S3Prefix",
    s3_input_mode = "File",
    s3_data_distribution_type = data_config.s3_data_distribution_type,
    s3_compression_type = data_config.s3_compression_type,
)

result_output = ProcessingOutput( 
    source = "/opt/ml/processing/output",
    destination = data_config.s3_output_path,
    output_name = "analysis_result",
    s3_upload_mode = "EndOfJob",
)

step_clarify = ProcessingStep(
    name = "ClarifyProcessingStep",
    processor = clarify_processor,
    inputs = [data_input, config_input],
    outputs = [result_output],
    depends_on = [step_config_file.name]
)

A etapa **RegisterChurnModel** é definida na variável chamada **step_register**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): registro de modelo – os trabalhos de registro são definidos usando a classe RegisterMode().
- **Estimator** (Estimador): o estimador é definido na variável **xgbtrain** no início na célula.
- **Model Data** (Dados do modelo): este é o URI do modelo que é retornado por **ChurnHyperParameterTuning**.
- **Content Types** (Tipos de conteúdo): texto/csv
- **Response Types** (Tipos de resposta) texto/csv
- **Inference Instance** (Instância de inferência): esse é o tipo de instância que será usado no processamento de inferência.
- **Transform Instance** (Instância de transformação): esse é o tipo de instância que será usado para processar transformações.
- **Model Package Group Name** (Nome do grupo de pacotes do modelo): esse é o nome do grupo que armazenará o grupo de versões do modelo.
- **Model Metrics** (Métricas do modelo): define o local das métricas do modelo. Os arquivos incluídos são o relatório de viés do SageMaker Clarify, o relatório de explicabilidade do SageMaker Clarify e a avaliação do modelo. 

In [ ]:
#configure-model-registry
model_statistics = MetricsSource(
    s3_uri = "s3://{}/output/evaluation/evaluation.json".format(default_bucket),
    content_type = "application/json"
    )
explainability = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    )

bias = MetricsSource(
    s3_uri = "s3://{}/clarify-output/bias/analysis.json".format(default_bucket),
    content_type = "application/json"
    ) 

model_metrics = ModelMetrics(
    model_statistics = model_statistics,
    explainability = explainability,
    bias = bias
)

step_register = RegisterModel(
    name = "RegisterChurnModel",
    estimator = xgb_train,
    model_data = step_tuning.get_top_model_s3_uri(top_k = 0, s3_bucket = default_bucket, prefix = "output"),
    content_types = ["text/csv"],
    response_types = ["text/csv"],
    inference_instances = ["ml.t2.medium", "ml.m5.large"],
    transform_instances = ["ml.m5.large"],
    model_package_group_name = model_package_group_name,
    model_metrics = model_metrics,
)

A etapa **CheckAUCScoreChurnEvaluation** é definida na variável chamada **step_cond**. 

A configuração de etapa inclui o seguinte:
- **Type** (Tipo): condição – os trabalhos de condição são definidos usando a classe ConditionStep().
- **Conditions** (Condições): essa condição é avaliada como True quando a saída de **ChurnEvalBestModel** é superior a 0,75.
- **If Steps** (Etapas if): essa é a lista de etapas que são executadas quando a condição é avaliada como True.
- **Else Steps** (Etapas else): essa é a lista de etapas que são executadas quando a condição é avaliada como False. Observe que esta lista está vazia, o que significa que o processamento do pipeline é interrompido quando a condição não é atendida.

In [ ]:
%%capture
cond_lte = ConditionGreaterThan(
    left = JsonGet(
        step = step_eval,
        property_file = evaluation_report,
        json_path = "binary_classification_metrics.auc.value"
    ),
    right = 0.75,
)

step_cond = ConditionStep(
    name = "CheckAUCScoreChurnEvaluation",
    conditions = [cond_lte],
    if_steps = [step_create_model, step_config_file, step_transform, step_clarify, step_register],
    else_steps = [],
)

### Tarefa 2.2.3 Definir o pipeline

Depois de definir as etapas, você vai configurar o pipeline na variável chamada **pipeline**. Observe como as etapas já definidas são passadas para a definição de pipeline.

In [ ]:
 #define pipeline function
def get_pipeline(
    region,
    role = None,
    default_bucket = None,
    model_package_group_name = "ChurnModelPackageGroup",
    pipeline_name = "ChurnModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version = None
    ):

    #configure pipeline instance
    pipeline = Pipeline(
        name = pipeline_name,
        parameters = [
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps = [step_process, step_tuning, step_eval, step_cond],
        sagemaker_session = sagemaker_session
    )
    return pipeline


### Tarefa 2.2.4 Criar o pipeline

In [ ]:
 #create pipeline using function
pipeline = get_pipeline(
  region = region,
    role = role,
    default_bucket = default_bucket,
    model_package_group_name = model_package_group_name,
    pipeline_name = pipeline_name,
    custom_image_uri = clarify_image,
    sklearn_processor_version = sklearn_processor_version
)

### Tarefa 2.2.5 Atualizar o pipeline para usar o perfil do IAM correto. 

In [ ]:
#set-iam-role
pipeline.upsert(role_arn = role)

**Observe:** se aparecer o aviso a seguir após a execução da célula, você poderá ignorá-lo com segurança.

"Não foi encontrado nenhum trabalho de treinamento concluído associado a este estimador. Verifique se este estimador é usado apenas para criar a configuração do fluxo de trabalho".

### Tarefa 2.2.6 Iniciar o pipeline

In [ ]:
#start-pipeline
RunPipeline = pipeline.start()

### Tarefa 2.2.7 Descrever o pipeline

In [ ]:
#describe-pipeline
RunPipeline.describe()

A execução deste pipeline exige cerca de 35 minutos.

Enquanto o pipeline estiver em execução, continue na próxima tarefa para explorar o pipeline no console do Amazon SageMaker Studio.

## Tarefa 2.3 Monitorar e aprovar o pipeline

Nesta tarefa, explore o pipeline usando o console do Amazon SageMaker Studio.

### Tarefa 2.3.1 Monitorar o pipeline no SageMaker Studio

A próxima tarefa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_10.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_10.ipynb** e selecione **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora as etapas do pipeline.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar as etapas do pipeline, retorne ao notebook selecionando a aba **lab_10.ipynb**.

1. No SageMaker Studio, selecione o ícone **Página inicial do SageMaker**.
1. Escolha **Pipelines**.

O SageMaker Studio abre a aba **Pipelines**.

1. Selecione o pipeline chamado **ChurnModelSMPipeline**. 

O SageMaker Studio abre a aba **ChurnModelSMPipeline**.

1. Na aba **ChurnModelSMPipeline**, em **Executions** (Execuções), abra (clique com o botão direito) o status do pipeline e selecione **Open execution details** (Abrir detalhes da execução). 

O SageMaker Studio abre a página **Grafo acíclico dirigido** (DAG).

O Grafo acíclico dirigido (DAG) mostra o fluxo de trabalho e o progresso do pipeline. As cores são usadas para indicar o status de uma etapa. Os indicadores de cor da etapa são:
- **cinza:** aguardando execução.
- **azul:** em execução.
- **verde:** concluído com sucesso.
- **vermelho:** erro.

Cada etapa explorada tem as mesmas quatro abas no painel de detalhes da etapa. Embora os títulos das abas sejam os mesmos, o conteúdo de cada uma varia de acordo com o tipo e a configuração do trabalho:

- **Input** (Entrada): essa aba contém as entradas que foram passadas para o trabalho. Exemplos de entradas são os parâmetros que especificam os tipos de instância, as funções do AWS Identity and Access Management (IAM) ou os argumentos necessários para executar o trabalho. Outros exemplos de entrada incluem arquivos, como código, conjuntos de dados e imagens do Docker.
- **Output** (Saída): essa aba mostra a saída que o trabalho criou. Alguns exemplos de saída são métricas, gráficos, arquivos e resultados de avaliação.
- **Logs:** essa aba contém uma lista de logs associados ao trabalho. Se o usuário tiver privilégios suficientes para os logs do Amazon CloudWatch, ele poderá clicar no link dos logs e visualizar as mensagem do log no CloudWatch. Alguns tipos de trabalho não geram logs.
- **Information** (Informações): essa aba contém informações básicas sobre um trabalho, como o tipo de trabalho, o nome do trabalho e quando ele foi executado.

1. Escolha a etapa chamada **ChurnModelProcess**. Um novo painel chamado **ChurnModelProcess** é exibido.
1. No painel **ChurnModelProcess**, examine as abas associadas a essa etapa do pipeline: 
    - Escolha a aba **Input** (Entrada). Essa aba contém informações úteis sobre os parâmetros e os arquivos que a etapa de processamento usa. Na lista de parâmetros, há detalhes, como o tipo de instância e a imagem que o trabalho usa, o local do conjunto de dados, o local do código e os destinos das diferentes saídas que são geradas. Role o painel até o final para encontrar as entradas de arquivo que foram passadas ao trabalho.
    - Escolha a aba **Output** (Saída). Essa aba mostra os diferentes arquivos que a etapa do pipeline gera e local em que eles são colocados. Este pipeline coloca todas as saídas no bucket padrão do SageMaker Studio.
    - Escolha a aba **Logs**. Essa aba mostra os logs que o trabalho gerou. A presença dos logs disponíveis dentro do SageMaker Studio acelera a investigação e a solução de problemas quando uma etapa do pipeline não é executada com sucesso.
    - Escolha a aba **Information** (Informações). Essa aba oferece uma visão geral abrangente da etapa do pipeline. Ela inclui informações, como o tipo de etapa, o nome da etapa e um link para log do trabalho. Ela também contém detalhes sobre quando o trabalho foi executado e a duração da execução.
        - Observe que o **Step Type** (Tipo de etapa) é **Processing** (Processamento).

### Tarefa 2.3.2 Descobrir os detalhes da etapa do pipeline

Nas próximas etapas, você vai escolher o nó apropriado do grafo acíclico dirigido (DAG) para encontrar informações sobre uma determinada etapa do pipeline. Se precisar de ajuda para encontrar as respostas, veja no final deste notebook Python as resposta e as dicas incluídas.

1. Para a etapa chamada **ChurnHyperParameterTuning**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Qual foi o **Melhor trabalho de treinamento em geral** gerado por esta etapa?
1. Para a etapa chamada **ChurnEvalBestModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Qual é o nome do script do Python usado para avaliar o principal modelo que foi identificado na etapa anterior?
    - Em que local este arquivo está?
    - Em que local os resultados desta etapa foram gravados?
1. Para a etapa chamada **CheckAUCScoreChurnEvaluation**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Qual foi o **Resultado da avaliação**?
1. Para a etapa chamada **ChurnCreateModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Este trabalho gerou logs?
1. Para a etapa chamada **RegisterChurnModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Qual é o valor da métrica área dentro da curva (AUC)?
1. Para a etapa chamada **ChurnTransform**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    - Este trabalho gerou logs?
    - Quais arquivos foram as entradas desta etapa?
1. Para a etapa chamada **ChurnModelConfigFile**, localize os seguintes detalhes: 
    - Qual ProcessingInstanceType foi usado para executar esse trabalho?
    - Qual é o **Tipo de etapa** desta etapa?
1. Para a etapa chamada **ClarifyProcessingStep**, localize os seguintes detalhes:
    - Qual foi a saída do arquivo desta etapa?
    - Em que local a saída foi gravada?

### 2.3.3 Aprovar o modelo no pipeline

1. Quando a execução do pipeline for concluída, visualize o modelo criado pelo pipeline no **Registro de modelo**:
    - No SageMaker Studio, selecione o ícone **Página inicial do SageMaker**.
    - Expanda a lista **Models** (Modelos).
    - Escolha **Model registry** (Registro de modelo).
    - Abra o Grupo de modelo chamado **ChurnModelPackageGroup**.
    - Na aba **ChurnModelPackageGroup**, abra (clique com o botão direito) a lista na tabela **Versions** (Versões) e escolha **Open model version** (Abrir versão do modelo). Observe que o status do modelo é **Pending** (Pendente). Observe também que o valor de **Execution** (Execução) é o nome da execução de pipeline que acabou de ser concluída.

    Há mais detalhes sobre o pipeline em cada aba:
    - **Activity** (Atividade): essa aba mostra a atividade do modelo. Ela contém informações do evento e o tempo decorrido desde que o modelo foi modificado.
    - **Model quality** (Qualidade do modelo): essa aba mostra as métricas de precisão do modelo.
    - **Explainability** (Explicabilidade): essa aba mostra a importância dos recursos do modelo em termos de valores Shapley (SHAP).
    - **Bias report** (Relatório de viés): essa guia mostra o possível viés do modelo.
    - **Inference recommender** (Recomendação de inferência): essa aba oferece recomendações para melhorar a relação preço/desempenho do modelo. Essa aba não contém dados porque esse recurso não é compatível com este pacote de modelos.
    - **Load test** (Teste de carga): nessa aba, é possível iniciar testes de carga para experimentar diferentes tipos de instância e avaliá-las em relação às métricas de throughput e latência necessárias para uma implantação de produção.
    - **Settings** (Configurações): essa aba mostra informações, como quando o modelo foi criado, qual pipeline gerou o modelo, o local em que o modelo está e o componente de teste associado ao modelo.

1. Aprovar o modelo. Esse processo é projetado para revisão manual antes da aprovação do modelo. No entanto, é possível automatizar a aprovação do modelo no pipeline:
    - Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status** (Atualizar status)</span>.
    - Abra a lista suspensa e escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">**Approved** (Aprovado)</span>.
    - Escolha <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">**Update status** (Atualizar status)</span>.
1. Feche a aba **ChurnModelPackageGroup**.

### Tarefa 2.3.4 Visualizar as etapas do pipeline usando o AWS SDK

Além de usar a IU do SageMaker Studio para visualizar detalhes do pipeline, você também pode usar comandos do AWS SDK. Por exemplo, o comando a seguir retorna uma lista das etapas do pipeline.

In [ ]:
#list-steps
RunPipeline.list_steps()

## Tarefa 2.4 Examinar os artefatos

A próxima tarefa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_10.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_10.ipynb** e escolha **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora os artefatos.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar os artefatos, retorne ao notebook selecionando a aba **lab_10.ipynb**.

### Tarefa 2.4.1 Examinar os artefatos no SageMaker Studio

Conforme o pipeline foi executado, cada etapa gerou artefatos, como arquivos, parâmetros treinados e modelos. É possível identificar os artefatos que o pipeline criou no SageMaker Studio.
1. Retorne à aba chamada **ChurnModelSMPipeline**.
1. Escolha a aba **Executions** (Execuções).
1. Abra (clique com o botão direito) a execução listada e escolha **View trial components generated by execution** (Visualizar os componentes de teste gerados pela execução). 

O SageMaker Studio abre uma nova aba chamada **Trial Component List** (Lista de componentes de teste). 

Uma lista de todos os trabalhos que o pipeline executou é exibida.

Observe que cada componente de teste tem um **Trial Component Type** (Tipo de componente de teste). As informações disponíveis nas várias abas no detalhe do teste associado dependem do tipo de componente de teste. Nem todas as abas no detalhe do teste são preenchidas com os dados de todos os tipos de componente.

1. Abra (clique com o botão direito) na primeira linha na lista de trabalhos e escolha **Open in trial details** (Abrir nos detalhes do teste). 

O SageMaker Studio abre uma nova aba chamada **Describe Trial Component** (Descrever componente de teste). 

Em **Trial Components** (Componentes de teste), há várias abas disponíveis. Dependendo do que uma etapa do pipeline fez, algumas abas podem estar vazias. 

1. Escolha a aba **Artifacts** (Artefatos). Os detalhes da entrada e da saída usados na etapa.
1. Escolha a aba **Explainability** (Explicabilidade). Essa aba mostra o relatório de explicabilidade que o SageMaker Clarify gerou.
1. Escolha a aba **Bias Report** (Relatório de viés). Essa aba mostra o relatório de viés que o SageMaker Clarify gerou.

### Tarefa 2.4.2 Localizar os artefatos no bucket do S3 padrão
**Atenção:** use o console de gerenciamento da AWS nesta tarefa. Depois de explorar o bucket do S3, retorne à aba do navegador em que o SageMaker Studio está aberto e escolha a aba **lab_10.ipynb**.

1. Na guia do navegador em que o console está aberto, navegue até o Amazon S3.
1. Escolha o nome do bucket que começa com **sagemaker-** e a região da AWS. Por exemplo: **sagemaker-us-west-2-123456789**.
1. Explore as pastas e os arquivos nesse bucket. Esse bucket contém o conjunto de dados, as entradas e as saídas do processamento, os resultados do SageMaker Clarify e outros arquivos que contribuíram para o modelo resultante.
1. Retorne à aba do navegador em que o SageMaker Studio está aberto e escolha a aba **lab_10.ipynb**.

## Tarefa 2.5 (opcional) Criar e examinar a linhagem do pipeline

Você aprendeu a usar o SageMaker Clarify para explicar como um modelo faz previsões e entender o possível viés de um modelo. Você também pode usar o SageMaker Clarify para descobrir as etapas usadas para gerar o modelo, que geralmente são necessárias para a auditoria do modelo. Nesta tarefa, você aproveita o módulo MLLineageHelper para criar a linhagem da execução de pipeline atual. Consulte [MLLineageHelper](https://github.com/aws-samples/ml-lineage-helper) para saber mais sobre o ML Lineage Helper.

O Amazon SageMaker ML Lineage Tracking cria e armazena informações sobre as etapas de um fluxo de trabalho de ML desde a preparação dos dados até a implantação do modelo. Com as informações de monitoramento, é possível reproduzir as etapas do fluxo de trabalho, monitorar a linhagem do modelo e do conjunto de dados e estabelecer os padrões de governança e auditoria do modelo.

### Tarefa 2.5.1 Configurar a sessão e as variáveis

In [ ]:
#set-variables
fs_query = feature_group.athena_query()
fs_table = fs_query.table_name
query_string = 'SELECT * FROM "'+fs_table+'"'

### Tarefa 2.5.2 Mostrar os valores que serão usados para criar a linhagem do modelo

As configurações incluem o seguinte:
- **query_string:** essa é a consulta do SageMaker Feature Store que será passada para o módulo MLLineageHelper.
- **model_ref:** esse é o nome do modelo que está sendo avaliado.
- **processing_job:** esse é o nome do trabalho de processamento que gerou o modelo.

In [ ]:
#print-values
print ('query_string:',query_string)

model_ref = sagemaker_client.list_models(SortBy = 'CreationTime', SortOrder = 'Descending')['Models'][0]['ModelName']
print ('model_ref:',model_ref)

processing_job = sagemaker_client.list_processing_jobs(SortBy = 'CreationTime', SortOrder = 'Descending', NameContains = 'ChurnModelProcess')['ProcessingJobSummaries'][0]['ProcessingJobName']
print ('processing_job:',processing_job)

processing_job_description = sagemaker_client.describe_processing_job(
    ProcessingJobName = processing_job
    )

### Tarefa 2.5.3 Descrever o trabalho de processamento

In [ ]:
#describe-processing-job
processing_job_description

### Tarefa 2.5.4 Mostrar o nome do trabalho de treinamento usado para criar o modelo

In [ ]:
#print-training-job
training_job_name  =  sagemaker_client.list_training_jobs(SortBy = 'CreationTime', SortOrder = 'Descending')['TrainingJobSummaries'][0]['TrainingJobName']
print (training_job_name)

### Tarefa 2.5.5 Criar a linhagem do modelo

Se ocorrer o erro a seguir, execute a célula novamente.
- **ClientError: ocorreu um erro (ThrottlingException) ao chamar a operação UpdateArtifact (máximo de novas tentativas atingido: 4): Taxa excedida**

In [ ]:
#build-lineage
ml_lineage = MLLineageHelper()
lineage = ml_lineage.create_ml_lineage(training_job_name, model_name = model_ref,
                                       query = query_string, sagemaker_processing_job_description = processing_job_description,
                                       feature_group_names = [feature_group_name])

### Tarefa 2.5.6 Limitar a linhagem para incluir somente o teste atual e o grupo de recursos

Um pipeline pode ser executado várias vezes. Para garantir que sejam recuperados os detalhes do trabalho de treinamento mais recentes, filtre a chamada de linhagem usando o nome do teste atual e o grupo de recursos que o teste usa. 

Após a execução dessa célula, as etapas usadas para criar o modelo, a ordem em que as etapas foram executadas e quais trabalhos contribuíram para outros trabalhos no pipeline são exibidos como uma tabela. Essas mesmas informações também são gravadas em um arquivo chamado **lineage_FS.csv**. Você pode baixar esse arquivo para salvar a saída e compartilhá-la com outros membros da equipe, como os auditores.

In [ ]:
#limit-lineage
trial_name = RunPipeline.describe()['PipelineExperimentConfig']['TrialName']
pat = str(trial_name)+'|'+'fg-FG'
df1 = lineage[lineage.apply(lambda x: any(x.str.contains(pat)),axis = 1)]
pd.set_option('display.max_colwidth', 120)
df1.to_csv('lineage_FS.csv') 
df1

### Tarefa 2.5.7 Gerar uma visualização da linhagem do modelo

In [ ]:
#visualize-lineage
plt.figure(3, figsize = (20, 14))
graph = nx.DiGraph()
graph.add_edges_from([(each[0], each[2]) for each in df1.values])
fig, ax = plt.subplots()
nx.draw_networkx(
    graph,
    node_size = 300,
    node_color = "orange",
    alpha = 0.65,
    font_size = 8,
    pos = nx.spring_layout(graph)
)
ax.set_facecolor('deepskyblue')
ax.axis('off')
fig.set_facecolor('deepskyblue')
plt.show()

## Tarefa 2.6 Remover o pipeline

Para excluir o pipeline, execute a seguinte célula:

In [ ]:
#delete-pipeline
response = sagemaker_client.delete_pipeline(PipelineName = 'ChurnModelSMPipeline')
print (response)

### Conclusão 

Parabéns! Você usou o SageMaker Pipelines para automatizar a criação e o registro de um modelo. Você aprendeu a fazer uma busca detalhada em cada etapa do pipeline para identificar os parâmetros, os arquivos e os logs associados. Você sabe como identificar os ativos que o pipeline usou para gerar o modelo, como encontrar o modelo no registro de modelo e como encontrar e visualizar os relatórios de explicabilidade e viés que um pipeline pode gerar.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.

### Dicas e respostas da Tarefa 2.3.1.1
Dica geral: o **Step type** (Tipo de etapa) se encontra na aba **Information** (Informações).

1. Para a etapa chamada **ChurnHyperParameterTuning**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** ajuste</br>
    - Qual foi o **Melhor trabalho de treinamento em geral** gerado por esta etapa? </br>
    **Dica:** essa informação se encontra na aba **Output** (Saída).</br>
    **Resposta:** o nome do modelo é gerado e será diferente para cada aluno. O nome deve ser semelhante a este exemplo: 056vhzs2vkxc-ChurnHy-TCAtUr16oV-001-17d5bd01
1. Para a etapa **ChurnEvalBestModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?
    **Resposta:** processamento</br>
    - Qual é o nome do script do Python usado para avaliar o principal modelo que foi identificado na etapa anterior?</br>
    **Dica:** essa informação se encontra na aba **Input** (Entrada).</br>
    **Resposta:** evaluate.py</br>
    - Em que local este arquivo está?</br>
    **Dica:** essa informação se encontra na aba **Input** (Entrada).</br>
    **Resposta:** o arquivo reside em um bucket do S3. O caminho é semelhante a este exemplo: s3://sagemaker-us-west-2-1234567890/input/code/evaluate.py</br>
    - Em que local os resultados desta etapa foram gravados?</br>
    **Dica:** essa informação se encontra na aba **Output** (Saída).</br>
    **Resposta:** os resultados da avaliação foram gravados em um bucket do S3. O caminho para o arquivo deve ser semelhante ao seguinte exemplo: s3://sagemaker-us-west-2-1234567890/output/evaluation</br>
1. Para a etapa **CheckAUCScoreChurnEvaluation**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** condição</br>
    - Qual foi o **Resultado da avaliação**?</br>
    **Dica:** essa informação se encontra na aba **Output** (Saída).</br>
    **Resposta:** verdadeiro
1. Para a etapa **ChurnCreateModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** modelo</br>
    - Este trabalho gerou logs?</br>
    **Resposta:** não
1. Para a etapa **RegisterChurnModel**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** RegisterModel</br>
    - Qual é o valor da métrica AUC?</br>
    **Dica:** essa informação se encontra na aba **Output** (Saída).</br>
    **Resposta:** o valor vai variar, mas deverá ser próximo de 0,98.
1. Para a etapa **ChurnTransform**, localize os seguintes detalhes:
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** transformação</br>
    - Este trabalho gerou logs?</br>
    **Resposta:** sim</br>
    - Quais arquivos foram as entradas desta etapa?</br>
    **Dica:** essa informação se encontra na aba **Input** (Entrada). Talvez seja necessário rolar o painel até o final para encontrar os nomes de arquivo.</br>
    **Resposta:** model.tar.gz, sagemaker-xgboost:1.5-1-cpu-py3, batch.csv
1. Para a etapa **ChurnModelConfigFile**, localize os seguintes detalhes: 
    - Qual ProcessingInstanceType foi usado para executar esse trabalho?</br>
    **Dica:** essa informação se encontra na aba **Input** (Entrada).</br>
    **Resposta:** ml.m5.xlarge
    - Qual é o **Tipo de etapa** desta etapa?</br>
    **Resposta:** processamento
1. Para a **ClarifyProcessingStep**, localize os seguintes detalhes:
    - Qual foi a saída do arquivo desta etapa?
    **Dica:** essa informação se encontra na aba **Output** (Saída).</br>
    **Resposta:** a saída foi dados de viés.
    - Em que local a saída foi gravada?
    **Resposta:** a saída foi gravada no bucket do S3. O caminho deve ser semelhante a este exemplo: s3://sagemaker-us-west-2-1234567890/clarify-output/bias